In [1]:
from IPython.display import display, HTML
import warnings
warnings.filterwarnings(action='ignore')
import datetime


# 데이터 전처리
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 기계학습 모델 및 평가
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, recall_score, precision_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm import tqdm
from sklearn.model_selection import KFold

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import ListedColormap
matplotlib.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
plt.rc('font', family='sans-serif')

In [12]:
import pickle

# pickle 파일 경로 설정
pickle_file_path = './data/train_전체데이터.pkl'  # 실제 파일 경로로 변경하세요

# pickle 파일 열기
with open(pickle_file_path, 'rb') as pickle_file:
    loaded_data = pickle.load(pickle_file)

# 열린 데이터 확인
print(loaded_data)

           date_w        상품 바코드                 상품명  판매수량  옵션코드_BX  옵션코드_CS  \
0      2021-09-13  7.920000e+04  한미6년근홍삼100ml*10(6)     4        1        0   
1      2021-09-20  7.920000e+04  한미6년근홍삼100ml*10(6)     1        1        0   
2      2022-01-31  7.920000e+04  한미6년근홍삼100ml*10(6)     3        1        0   
3      2021-09-27  4.133300e+10       건전지]듀라셀<MN21>     1        0        0   
4      2021-10-04  4.133300e+10       건전지]듀라셀<MN21>     1        0        0   
...           ...           ...                 ...   ...      ...      ...   
182140 2022-04-25  8.880106e+13         롯데]칸쵸컵<88g>     2        1        0   
182141 2022-05-02  8.880106e+13         롯데]칸쵸컵<88g>     1        1        0   
182142 2022-05-09  8.880106e+13         롯데]칸쵸컵<88g>     2        1        0   
182143 2022-05-30  8.880106e+13         롯데]칸쵸컵<88g>     3        1        0   
182144 2022-06-13  8.880106e+13         롯데]칸쵸컵<88g>     1        1        0   

        옵션코드_EA       공휴일  평균기온(°C)  최저기온(°C)  최고기온

In [15]:
train_gp = loaded_data.copy()
train_gp

,date_w,상품 바코드,상품명,판매수량,옵션코드_BX,옵션코드_CS,옵션코드_EA,공휴일,평균기온(°C),최저기온(°C),최고기온(°C),patient,년도,월,일,주기성_변수,월_일_년도,판매수량_가중치
0,2021-09-13,7.920000e+04,한미6년근홍삼100ml*10(6),4,1,0,0,0.000000,0.037080,0.034931,0.041153,0.044795,2021,9,13,-1.000000e+00,93321,0.001584
1,2021-09-20,7.920000e+04,한미6년근홍삼100ml*10(6),1,1,0,0,0.004751,0.037103,0.032872,0.042488,0.068324,2021,9,20,-1.000000e+00,94021,0.001584
2,2022-01-31,7.920000e+04,한미6년근홍삼100ml*10(6),3,1,0,0,0.004751,0.001312,-0.004118,0.009050,1.651764,2022,1,31,5.000000e-01,15122,0.001584
3,2021-09-27,4.133300e+10,건전지]듀라셀<MN21>,1,0,0,1,0.002375,0.053992,0.047035,0.064885,0.169339,2021,9,27,-1.000000e+00,94721,0.002375
4,2021-10-04,4.133300e+10,건전지]듀라셀<MN21>,1,0,0,1,0.004751,0.056469,0.048935,0.067193,0.131671,2021,10,4,-8.660254e-01,102421,0.002375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182140,2022-04-25,8.880106e+13,롯데]칸쵸컵<88g>,2,1,0,0,0.000000,0.004634,0.003669,0.005838,1.094063,2022,4,25,8.660254e-01,44522,0.000305
182141,2022-05-02,8.880106e+13,롯데]칸쵸컵<88g>,1,1,0,0,0.000611,0.005389,0.003778,0.007064,0.823460,2022,5,2,5.000000e-01,52222,0.000305
182142,2022-05-09,8.880106e+13,롯데]칸쵸컵<88g>,2,1,0,0,0.000000,0.005066,0.003857,0.006431,0.725201,2022,5,9,5.000000e-01,52922,0.000305
182143,2022-05-30,8.880106e+13,롯데]칸쵸컵<88g>,3,1,0,0,0.000305,0.006178,0.005066,0.007544,0.264888,2022,5,30,5.000000e-01,55022,0.000305


In [16]:
# 가중치 적용
train_gp['공휴일'] = train_gp['판매수량_가중치'] * train_gp['공휴일']
train_gp['평균기온(°C)'] = train_gp['판매수량_가중치'] * train_gp['평균기온(°C)']
train_gp['최저기온(°C)'] = train_gp['판매수량_가중치'] * train_gp['최저기온(°C)']
train_gp['최고기온(°C)'] = train_gp['판매수량_가중치'] * train_gp['최고기온(°C)']
train_gp['patient'] = train_gp['판매수량_가중치'] * train_gp['patient']


train_gp['년도'] = train_gp['판매수량_가중치'] * train_gp['년도']
train_gp['월'] = train_gp['판매수량_가중치'] * train_gp['월']
train_gp['일'] = train_gp['판매수량_가중치'] * train_gp['일']
# train_new['주기성_변수'] = train_new['판매수량_가중치'] * train_new['주기성_변수']
train_gp['월_일_년도'] = train_gp['판매수량_가중치'] * train_gp['월_일_년도']

In [17]:
train_gp.corr()['판매수량'].sort_values(ascending = False)

판매수량        1.000000
년도          0.787949
판매수량_가중치    0.787943
일           0.694469
월_일_년도      0.688699
월           0.677409
최고기온(°C)    0.648326
평균기온(°C)    0.628302
최저기온(°C)    0.587470
공휴일         0.250563
patient     0.228025
옵션코드_EA     0.059331
옵션코드_CS     0.023958
주기성_변수     -0.000827
상품 바코드     -0.047723
옵션코드_BX    -0.079256
Name: 판매수량, dtype: float64

In [18]:
train_add = train_gp.copy()
train_add = train_add[['상품 바코드', 'date_w', '상품명', '공휴일','최고기온(°C)', 'patient', '년도', '월', '일', '월_일_년도','판매수량']]
train_add = train_add.sort_values(['상품명', 'date_w'])
train_add

,상품 바코드,date_w,상품명,공휴일,최고기온(°C),patient,년도,월,일,월_일_년도,판매수량
13766,2.838022e+12,2021-01-04,감열지(전산용지)(25) 2입*1,0.000000,0.000233,0.003463,23.261059,0.011510,0.046039,142.961711,10
13767,2.838022e+12,2021-01-11,감열지(전산용지)(25) 2입*1,0.000000,0.001215,0.002101,23.261059,0.011510,0.126606,151.018486,10
13768,2.838022e+12,2021-01-18,감열지(전산용지)(25) 2입*1,0.000000,0.001347,0.001571,23.261059,0.011510,0.207174,159.075260,5
13769,2.838022e+12,2021-02-01,감열지(전산용지)(25) 2입*1,0.000000,0.001450,0.000908,23.261059,0.023019,0.011510,254.605588,33
13770,2.838022e+12,2021-02-08,감열지(전산용지)(25) 2입*1,0.000397,0.001633,0.000530,23.261059,0.023019,0.092077,262.662363,25
...,...,...,...,...,...,...,...,...,...,...,...
95146,8.801746e+12,2022-05-09,희창]율무차<900g*12>,0.000000,0.000373,0.042045,8.508632,0.021040,0.037872,222.697238,3
95147,8.801746e+12,2022-05-16,희창]율무차<900g*12>,0.000000,0.000489,0.030475,8.508632,0.021040,0.067328,225.642857,7
95148,8.801746e+12,2022-05-23,희창]율무차<900g*12>,0.000000,0.000503,0.023951,8.508632,0.021040,0.096785,228.588476,6
95149,8.801746e+12,2022-05-30,희창]율무차<900g*12>,0.000018,0.000437,0.015357,8.508632,0.021040,0.126241,231.534096,2


In [19]:
#시계열 데이터를 지도학습 형식으로 변환하여, 기계 학습 모델에 활용할 수 있도록 변환
def series_to_supervised(data, window=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]

    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    agg = agg[agg['상품 바코드(t-%d)' % window] == agg['상품 바코드(t)']]

    return agg

In [20]:
window = 10
series = series_to_supervised(train_add, window=window)

In [21]:
series['date_t_check'] = series['date_w(t-10)'] + datetime.timedelta(weeks=10)
series = series[series['date_t_check'] == series['date_w(t)']]
series.drop('date_t_check', axis=1, inplace=True)

In [22]:
columns_to_drop = []

for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['상품 바코드', '상품명', 'date_w', '공휴일',
       '최고기온(°C)', 'patient', '년도', '월', '일', '월_일_년도']] #최고기온 제외

series.drop(columns_to_drop, axis=1, inplace=True)

In [23]:
series.columns

Index(['판매수량(t-10)', '판매수량(t-9)', '판매수량(t-8)', '판매수량(t-7)', '판매수량(t-6)',
       '판매수량(t-5)', '판매수량(t-4)', '판매수량(t-3)', '판매수량(t-2)', '판매수량(t-1)',
       '상품 바코드(t)', 'date_w(t)', '상품명(t)', '공휴일(t)', '최고기온(°C)(t)',
       'patient(t)', '년도(t)', '월(t)', '일(t)', '월_일_년도(t)', '판매수량(t)'],
      dtype='object')

In [24]:
# 열 이름 변경 - (t) 제거
columns_to_modify = ['상품 바코드(t)', 'date_w(t)', '상품명(t)', '판매수량(t)', '공휴일(t)',
      '최고기온(°C)(t)', 'patient(t)', '년도(t)',
       '월(t)', '일(t)', '월_일_년도(t)']
new_column_names = {col : col[:-3] if len(col) > 3 else col for col in columns_to_modify}

# 열 이름 변경을 위한 복사본 생성
series = series.rename(columns = new_column_names)

series

,판매수량(t-10),판매수량(t-9),판매수량(t-8),판매수량(t-7),판매수량(t-6),판매수량(t-5),판매수량(t-4),판매수량(t-3),판매수량(t-2),판매수량(t-1),...,date_w,상품명,공휴일,최고기온(°C),patient,년도,월,일,월_일_년도,판매수량
13786,14.0,6.0,43.0,5.0,4.0,8.0,10.0,5.0,25.0,10.0,...,2021-06-28,감열지(전산용지)(25) 2입*1,0.000000,0.003571,0.000681,23.261059,0.069058,0.322271,746.068841,8
13787,6.0,43.0,5.0,4.0,8.0,10.0,5.0,25.0,10.0,8.0,...,2021-07-05,감열지(전산용지)(25) 2입*1,0.000000,0.003704,0.001236,23.261059,0.080568,0.057548,834.693362,4
13788,43.0,5.0,4.0,8.0,10.0,5.0,25.0,10.0,8.0,4.0,...,2021-07-12,감열지(전산용지)(25) 2입*1,0.000000,0.004063,0.001838,23.261059,0.080568,0.138116,842.750136,7
13789,5.0,4.0,8.0,10.0,5.0,25.0,10.0,8.0,4.0,7.0,...,2021-07-19,감열지(전산용지)(25) 2입*1,0.000000,0.004067,0.002971,23.261059,0.080568,0.218684,850.806911,8
13790,4.0,8.0,10.0,5.0,25.0,10.0,8.0,4.0,7.0,8.0,...,2021-07-26,감열지(전산용지)(25) 2입*1,0.000000,0.004286,0.003520,23.261059,0.080568,0.299252,858.863686,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75213,12.0,7.0,10.0,8.0,2.0,7.0,6.0,11.0,4.0,2.0,...,2021-12-13,후루트링320g<12>,0.000000,0.000163,0.003024,8.581581,0.050954,0.055201,523.646283,3
75214,7.0,10.0,8.0,2.0,7.0,6.0,11.0,4.0,2.0,3.0,...,2021-12-20,후루트링320g<12>,0.000018,0.000169,0.002316,8.581581,0.050954,0.084924,526.618627,9
123809,10.0,35.0,114.0,361.0,226.0,113.0,134.0,50.0,21.0,110.0,...,2021-10-11,휴대용 우의 (50),0.003933,0.085008,0.142710,126.743349,0.627132,0.689845,6467.046438,35
123810,35.0,114.0,361.0,226.0,113.0,134.0,50.0,21.0,110.0,35.0,...,2021-10-18,휴대용 우의 (50),0.000000,0.072198,0.153947,126.743349,0.627132,1.128837,6510.945668,6


In [25]:
series.drop('상품명', axis=1, inplace = True)

# MultiIndex를 설정할 열들
index_columns = ['date_w', '상품 바코드']

# MultiIndex로 설정
series.set_index(index_columns, inplace=True)
series.sort_index(ascending=True, inplace=True)

In [26]:
series

판매수량(t-10)  판매수량(t-9)  판매수량(t-8)  판매수량(t-7)  \
date_w     상품 바코드                                                      
2021-03-15 1.701001e+12        12.0       22.0       36.0       13.0   
           1.701001e+12        11.0        3.0        5.0        3.0   
           1.701002e+12         8.0       16.0       12.0       13.0   
           1.701002e+12       137.0      135.0       61.0      149.0   
           1.701002e+12       133.0      134.0      118.0      137.0   
...                             ...        ...        ...        ...   
2022-06-27 6.880106e+13        11.0        7.0        4.0        7.0   
           6.880106e+13         9.0        7.0        5.0        7.0   
           6.880106e+13        10.0        3.0        8.0       13.0   
           6.880106e+13         6.0        1.0        4.0        1.0   
           7.880106e+13         1.0        2.0        5.0        4.0   

                         판매수량(t-6)  판매수량(t-5)  판매수량(t-4)  판매수량(t-3)  \
date_w     상품 바코드                                                     
2021-03-15 1.701001e+12       24.0       26.0       20.0       33.0   
           1.701001e+12        3.0       23.0       10.0       11.0   
           1.701002e+12       15.0        4.0        8.0       12.0   
           1.701002e+12      146.0       97.0       85.0      135.0   
           1.701002e+12      149.0      115.0      116.0      116.0   
...                            ...        ...        ...        ...   
2022-06-27 6.880106e+13       12.0       16.0        4.0        7.0   
           6.880106e+13        8.0        9.0       11.0        8.0   
           6.880106e+13       11.0       67.0        8.0        4.0   
           6.880106e+13        9.0        9.0        7.0        6.0   
           7.880106e+13        3.0        5.0        4.0        5.0   

                         판매수량(t-2)  판매수량(t-1)  공휴일  최고기온(°C)   patient  \
date_w     상품 바코드                                                        
2021-03-15 1.701001e+12       29.0       20.0  0.0  0.128863  0.041681   
           1.701001e+12       12.0       18.0  0.0  0.000528  0.000171   
           1.701002e+12       10.0       13.0  0.0  0.002036  0.000659   
           1.701002e+12      123.0      118.0  0.0  0.175003  0.056605   
           1.701002e+12      125.0      111.0  0.0  0.192191  0.062164   
...                            ...        ...  ...       ...       ...   
2022-06-27 6.880106e+13       16.0        8.0  0.0  0.001483  0.025887   
           6.880106e+13       17.0        7.0  0.0  0.000936  0.016326   
           6.880106e+13       15.0        8.0  0.0  0.001613  0.028154   
           6.880106e+13        1.0        6.0  0.0  0.000525  0.009161   
           7.880106e+13        5.0        3.0  0.0  0.001065  0.018585   

                                 년도         월         일       월_일_년도  판매수량  
date_w     상품 바코드                                                           
2021-03-15 1.701001e+12  181.941209  0.270076  1.350380  3017.739377    26  
           1.701001e+12   11.650146  0.017294  0.086468   193.233325     7  
           1.701002e+12   22.871906  0.033951  0.169757   379.361283     9  
           1.701002e+12  212.026752  0.314735  1.573677  3516.748514    76  
           1.701002e+12  222.194779  0.329829  1.649145  3685.398901   114  
...                             ...       ...       ...          ...   ...  
2022-06-27 6.880106e+13   14.338089  0.042546  0.191458   458.946496     5  
           6.880106e+13   11.386551  0.033788  0.152046   364.471010     6  
           6.880106e+13   14.952757  0.044370  0.199666   478.621333     6  
           6.880106e+13    8.529425  0.025310  0.113894   273.017536     3  
           7.880106e+13   12.148553  0.036049  0.162221   388.861836     4  

[25475 rows x 18 columns]

In [27]:
series_y = series[['판매수량']]
series_y

판매수량
date_w     상품 바코드            
2021-03-15 1.701001e+12    26
           1.701001e+12     7
           1.701002e+12     9
           1.701002e+12    76
           1.701002e+12   114
...                       ...
2022-06-27 6.880106e+13     5
           6.880106e+13     6
           6.880106e+13     6
           6.880106e+13     3
           7.880106e+13     4

[25475 rows x 1 columns]

In [28]:
series_x = series.copy()
series_x.drop('판매수량', axis=1, inplace = True)
series_x

판매수량(t-10)  판매수량(t-9)  판매수량(t-8)  판매수량(t-7)  \
date_w     상품 바코드                                                      
2021-03-15 1.701001e+12        12.0       22.0       36.0       13.0   
           1.701001e+12        11.0        3.0        5.0        3.0   
           1.701002e+12         8.0       16.0       12.0       13.0   
           1.701002e+12       137.0      135.0       61.0      149.0   
           1.701002e+12       133.0      134.0      118.0      137.0   
...                             ...        ...        ...        ...   
2022-06-27 6.880106e+13        11.0        7.0        4.0        7.0   
           6.880106e+13         9.0        7.0        5.0        7.0   
           6.880106e+13        10.0        3.0        8.0       13.0   
           6.880106e+13         6.0        1.0        4.0        1.0   
           7.880106e+13         1.0        2.0        5.0        4.0   

                         판매수량(t-6)  판매수량(t-5)  판매수량(t-4)  판매수량(t-3)  \
date_w     상품 바코드                                                     
2021-03-15 1.701001e+12       24.0       26.0       20.0       33.0   
           1.701001e+12        3.0       23.0       10.0       11.0   
           1.701002e+12       15.0        4.0        8.0       12.0   
           1.701002e+12      146.0       97.0       85.0      135.0   
           1.701002e+12      149.0      115.0      116.0      116.0   
...                            ...        ...        ...        ...   
2022-06-27 6.880106e+13       12.0       16.0        4.0        7.0   
           6.880106e+13        8.0        9.0       11.0        8.0   
           6.880106e+13       11.0       67.0        8.0        4.0   
           6.880106e+13        9.0        9.0        7.0        6.0   
           7.880106e+13        3.0        5.0        4.0        5.0   

                         판매수량(t-2)  판매수량(t-1)  공휴일  최고기온(°C)   patient  \
date_w     상품 바코드                                                        
2021-03-15 1.701001e+12       29.0       20.0  0.0  0.128863  0.041681   
           1.701001e+12       12.0       18.0  0.0  0.000528  0.000171   
           1.701002e+12       10.0       13.0  0.0  0.002036  0.000659   
           1.701002e+12      123.0      118.0  0.0  0.175003  0.056605   
           1.701002e+12      125.0      111.0  0.0  0.192191  0.062164   
...                            ...        ...  ...       ...       ...   
2022-06-27 6.880106e+13       16.0        8.0  0.0  0.001483  0.025887   
           6.880106e+13       17.0        7.0  0.0  0.000936  0.016326   
           6.880106e+13       15.0        8.0  0.0  0.001613  0.028154   
           6.880106e+13        1.0        6.0  0.0  0.000525  0.009161   
           7.880106e+13        5.0        3.0  0.0  0.001065  0.018585   

                                 년도         월         일       월_일_년도  
date_w     상품 바코드                                                     
2021-03-15 1.701001e+12  181.941209  0.270076  1.350380  3017.739377  
           1.701001e+12   11.650146  0.017294  0.086468   193.233325  
           1.701002e+12   22.871906  0.033951  0.169757   379.361283  
           1.701002e+12  212.026752  0.314735  1.573677  3516.748514  
           1.701002e+12  222.194779  0.329829  1.649145  3685.398901  
...                             ...       ...       ...          ...  
2022-06-27 6.880106e+13   14.338089  0.042546  0.191458   458.946496  
           6.880106e+13   11.386551  0.033788  0.152046   364.471010  
           6.880106e+13   14.952757  0.044370  0.199666   478.621333  
           6.880106e+13    8.529425  0.025310  0.113894   273.017536  
           7.880106e+13   12.148553  0.036049  0.162221   388.861836  

[25475 rows x 17 columns]

In [29]:
# 연속형 데이터 스케일링 (standardScaler)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
columns_to_scale = series_x.columns

scaler = StandardScaler()
series_x[columns_to_scale] = scaler.fit_transform(series_x[columns_to_scale])

# 정규화
# scaler = MinMaxScaler()
# series_x[columns_to_scale] = scaler.fit_transform(series_x[columns_to_scale])

series_x

판매수량(t-10)  판매수량(t-9)  판매수량(t-8)  판매수량(t-7)  \
date_w     상품 바코드                                                      
2021-03-15 1.701001e+12   -0.147661   0.008362   0.223935  -0.128385   
           1.701001e+12   -0.163300  -0.285009  -0.253933  -0.279700   
           1.701002e+12   -0.210219  -0.084281  -0.146028  -0.128385   
           1.701002e+12    1.807263   1.753153   0.609313   1.929508   
           1.701002e+12    1.744705   1.737713   1.487975   1.747930   
...                             ...        ...        ...        ...   
2022-06-27 6.880106e+13   -0.163300  -0.223247  -0.269349  -0.219174   
           6.880106e+13   -0.194579  -0.223247  -0.253933  -0.219174   
           6.880106e+13   -0.178940  -0.285009  -0.207688  -0.128385   
           6.880106e+13   -0.241497  -0.315891  -0.269349  -0.309963   
           7.880106e+13   -0.319694  -0.300450  -0.253933  -0.264569   

                         판매수량(t-6)  판매수량(t-5)  판매수량(t-4)  판매수량(t-3)  \
date_w     상품 바코드                                                     
2021-03-15 1.701001e+12   0.035143   0.065872  -0.023295   0.169931   
           1.701001e+12  -0.272179   0.021464  -0.171030  -0.155887   
           1.701002e+12  -0.096567  -0.259789  -0.200577  -0.141078   
           1.701002e+12   1.820537   1.116869   0.936982   1.680542   
           1.701002e+12   1.864440   1.383319   1.394960   1.399153   
...                            ...        ...        ...        ...   
2022-06-27 6.880106e+13  -0.140470  -0.082156  -0.259671  -0.215127   
           6.880106e+13  -0.199007  -0.185775  -0.156256  -0.200317   
           6.880106e+13  -0.155104   0.672786  -0.200577  -0.259557   
           6.880106e+13  -0.184373  -0.185775  -0.215350  -0.229937   
           7.880106e+13  -0.272179  -0.244986  -0.259671  -0.244747   

                         판매수량(t-2)  판매수량(t-1)       공휴일  최고기온(°C)   patient  \
date_w     상품 바코드                                                             
2021-03-15 1.701001e+12   0.110620  -0.022889 -0.034054  0.052136 -0.030468   
           1.701001e+12  -0.141009  -0.052226 -0.034054 -0.069602 -0.030668   
           1.701002e+12  -0.170612  -0.125568 -0.034054 -0.068172 -0.030666   
           1.701002e+12   1.501980   1.414605 -0.034054  0.095904 -0.030396   
           1.701002e+12   1.531584   1.311926 -0.034054  0.112208 -0.030369   
...                            ...        ...       ...       ...       ...   
2022-06-27 6.880106e+13  -0.081802  -0.198909 -0.034054 -0.068696 -0.030544   
           6.880106e+13  -0.067000  -0.213577 -0.034054 -0.069216 -0.030590   
           6.880106e+13  -0.096604  -0.198909 -0.034054 -0.068573 -0.030533   
           6.880106e+13  -0.303827  -0.228246 -0.034054 -0.069605 -0.030625   
           7.880106e+13  -0.244621  -0.272251 -0.034054 -0.069093 -0.030579   

                               년도         월         일    월_일_년도  
date_w     상품 바코드                                                
2021-03-15 1.701001e+12  1.234252  0.416716  0.952640  0.460534  
           1.701001e+12 -0.237606 -0.268790 -0.213398 -0.269130  
           1.701002e+12 -0.140614 -0.223616 -0.136559 -0.221047  
           1.701002e+12  1.494287  0.537825  1.158646  0.589445  
           1.701002e+12  1.582172  0.578756  1.228270  0.633013  
...                           ...       ...       ...       ...  
2022-06-27 6.880106e+13 -0.214373 -0.200309 -0.116538 -0.200487  
           6.880106e+13 -0.239884 -0.224060 -0.152898 -0.224893  
           6.880106e+13 -0.209061 -0.195362 -0.108966 -0.195404  
           6.880106e+13 -0.264579 -0.247051 -0.188096 -0.248519  
           7.880106e+13 -0.233298 -0.217928 -0.143511 -0.218592  

[25475 rows x 17 columns]

In [30]:
# 랜덤으로 자르기
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(series_x, series_y, test_size=0.3, random_state=0)

# 데이터 컬럼 단위 정규화 하기
normalizer = StandardScaler()
X_train_scaled = normalizer.fit_transform(X_train)
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)

X_val_scaled = normalizer.transform(X_val.iloc[:,:])
X_val_df = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)

print('train 데이터:', X_train.shape)
print('val 데이터:', X_val.shape)

train 데이터: (17832, 17)
val 데이터: (7643, 17)


In [33]:
rfc = RandomForestClassifier(n_estimators=10,random_state=0)
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

In [69]:
#     'n_estimators': [50, 60, 70],
#     'max_depth': [10,15,20],
#     'max_leaf_nodes': [50, 100, 200],
#     'criterion':['gini', 'entropy'],
#     'max_features':['auto','sqrt', 'log2']

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [60, 65],
    'max_depth': [2, 3, 4],
    'max_leaf_nodes': [26, 28, 30],
    'max_features':['auto','sqrt', 'log2']
}

rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 2, n_jobs = -1)
grid_cv.fit(X_train,y_train)

print('최적의 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도:{0:.4f}'.format(grid_cv.best_score_))

최적의 하이퍼 파라미터:
 {'max_depth': 4, 'max_features': 'sqrt', 'max_leaf_nodes': 26, 'n_estimators': 60}
최고 예측 정확도:0.1115


In [62]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [60, 65],
    'max_depth': [2, 3, 4],
    'max_leaf_nodes': [26, 28, 30],
    'criterion':['friedman_mse', 'squared_error', 'poisson', 'absolute_error'],
    'max_features':['auto','sqrt', 'log2']
}

rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 2, n_jobs = -1)
grid_cv.fit(X_train,y_train)

print('최적의 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도:{0:.4f}'.format(grid_cv.best_score_))

ValueError: 
All the 432 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\user\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\user\AppData\Local\anaconda3\lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_parameter_constraints
    raise InvalidParameterError(
s

In [1]:
# train_test_split (랜덤)
# 하이퍼파라미터 설정
criterion = 'absolute_error'
max_depth = 4
max_features = 'sqrt'
max_leaf_nodes = 26
n_estimators = 39
random_state = 0

# RandomForestRegressor 모델 생성
rf_model = RandomForestRegressor(
    criterion=criterion,
    max_depth=max_depth,
    max_features=max_features,
    max_leaf_nodes=max_leaf_nodes,
    n_estimators=n_estimators,
    random_state=random_state)

#rf_model = RandomForestRegressor(random_state=11)

rf_model.fit(X_train_df, y_train)

rf_train_pred = rf_model.predict(X_train_df)
rf_valid_pred = rf_model.predict(X_val_df)

train_rmse = mean_squared_error(y_train, rf_train_pred, squared=False)
valid_rmse = mean_squared_error(y_val, rf_valid_pred, squared=False)

print('Train rmse:', train_rmse)
print('Validation rmse:', valid_rmse)
print()
print("훈련 세트 정확도 (r2): {:.3f}".format(rf_model.score(X_train_df, y_train)))
print("테스트 세트 정확도 (r2): {:.3f}".format(rf_model.score(X_val_df, y_val)))

date_w_values = sorted(X_val_df.index.get_level_values('date_w')) # 다중 인덱스라서 추가
plt.figure(figsize=(10, 6))
plt.title('Random Forest')
plt.plot(date_w_values, y_val)
plt.plot(date_w_values, rf_valid_pred) # 모델명
plt.legend(["y_real", "y_pred"])
plt.xticks(fontsize=8)
plt.show()

NameError: name 'RandomForestRegressor' is not defined